In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime 
import pytz
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.moneycontrol.com/stocks/fno/view_option_chain.php?ind_id=9')
main_page = driver.current_window_handle
time.sleep(5)

In [3]:
strike = int(input("Please Enter Nifty Strike : "))

Please Enter Nifty Strike : 12800


In [4]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)

client = gspread.authorize(creds)

sheet1 = client.open("options").worksheet('Nifty_Data')
sheet2 = client.open("options").worksheet('Nifty_Chain')

In [5]:
IST = pytz.timezone('Asia/Kolkata')
n_data = pd.DataFrame(columns = ['time','chng_oi_ce','chng_oi_pe','data'])
#n_data = n_data.append({'time':'10:00','chng_oi_ce':7500,'chng_oi_pe':4500,'data':-3000},ignore_index = True)

def chain_save():
    global driver,df
    p = driver.find_element_by_tag_name('html')
    page = p.get_attribute('outerHTML')
    aa = pd.read_html(page)
    df = aa[1].rename(columns = {0:'ltp_ce', 1:'net_chng_ce', 2:'vol_ce',3:'oi_ce',4:'chng_oi_ce',5:'strike',6:'ltp_pe',7:'net_chng_pe',8:'vol_pe',9:'oi_pe',10:'chng_oi_pe'})
    #df.to_csv('oi_data.csv',index = False)

def get_data():
    global df,IST,data,ce_sum,pe_sum,strike
    
    df.index[df['strike'] == strike][0]
    lower = df.index[df['strike'] == strike][0] - 12
    upper = df.index[df['strike'] == strike][0] + 12

    df1 = df.loc[lower:upper,['chng_oi_ce','strike','chng_oi_pe']]
    
    
    ce_sum = 0
    for a in df1['chng_oi_ce']:
        if a != '-':
            if int(a)>10000 :
                ce_sum = ce_sum + int(a)
                #print(a)
            elif int(a)<-10000 and int(a)<0:
                ce_sum = ce_sum + int(a)
                
            
    print('Change in Call Open Interest : ',ce_sum)

    pe_sum = 0
    for b in df1['chng_oi_pe']:
        
        if b != '-':
            if int(b)>10000 or (int(b)<10000 and int(b)<0):
                pe_sum = pe_sum + int(b)
                #print(b)
            elif int(b)<10000 and int(b)<0:
                pe_sum = pe_sum + int(b)
    print('Change in Put Open Interest : ',pe_sum)
    
    data = pe_sum-ce_sum
    
    datetime_ist = datetime.now(IST) 
    
    if int(datetime_ist.strftime('%M'))%2 == 1:
        print("Time : ",datetime_ist.strftime('%H:%M'),'\t',"DATA : ",data)
        print('\n')
        
def save_chain():
    global df,IST,data,ce_sum,pe_sum,strike,nifty_chain
    
    lower_ = df.index[df['strike'] == strike][0] - 6
    upper_ = df.index[df['strike'] == strike][0] + 6
    
    nifty_chain = df.loc[lower_:upper_,['oi_ce','chng_oi_ce','strike','chng_oi_pe','oi_pe']]
    #nifty_chain.to_csv('nifty_chain.csv',index = False)
    
    #print(nifty_chain)

def save_data():
    global df,IST,data,ce_sum,pe_sum,strike,n_data,c_time
    datetime_ist = datetime.now(IST) 
    c_time = datetime_ist.strftime('%H:%M')
    m_time = int(datetime_ist.strftime('%M'))
    '''if m_time%10 != 0:
        ind = 0
    if m_time%10 == 0:
        ind = 1
    if ind == 1:
        hello = 1
    elif ind ==0:
        n_data.drop(df.tail(1).index,inplace=True)
    '''
    if m_time%15 == 0:
        n_data = n_data.append({'time':c_time,'chng_oi_ce':ce_sum,'chng_oi_pe':pe_sum,'data':data},ignore_index = True)
        #n_data.to_csv('nifty_data',index = False)
    
def upload_data():
    global nifty_chain,n_data,c_time
    
    l = []
    for a_,b_ in nifty_chain.iterrows():
        l.append(b_.to_list())

    sheet2.batch_update([{ 
        'range' : 'A2:E14',
        'values' : l,
    }])
    
    
    l1 = []
    for a1,b1 in n_data.iterrows():
        l1.append(b1.to_list())
        
    l1.append([c_time,ce_sum,pe_sum,data])
        
    sheet1.batch_update([{ 
        'range' : 'A2:D30',
        'values' : l1,
    }])
    
    

In [ ]:
while True:
    try:
        chain_save()
    except:
        time.sleep(2)
        chain_save()
    
    get_data()
    
    save_chain()
    
    save_data()
    
    upload_data()

    time.sleep(60)

Change in Call Open Interest :  3390225
Change in Put Open Interest :  4809675
Time :  10:35 	 DATA :  1419450


Change in Call Open Interest :  3390225
Change in Put Open Interest :  4809675
Time :  10:37 	 DATA :  1419450


Change in Call Open Interest :  3438300
Change in Put Open Interest :  4809225
Change in Call Open Interest :  3438300
Change in Put Open Interest :  4809225
Time :  10:39 	 DATA :  1370925


Change in Call Open Interest :  3518925
Change in Put Open Interest :  4817175
Change in Call Open Interest :  3518925
Change in Put Open Interest :  4817175
Time :  10:41 	 DATA :  1298250


Change in Call Open Interest :  3517800
Change in Put Open Interest :  4820175
Change in Call Open Interest :  3517800
Change in Put Open Interest :  4820175
Time :  10:43 	 DATA :  1302375


Change in Call Open Interest :  3557325
Change in Put Open Interest :  4779225
Change in Call Open Interest :  3557325
Change in Put Open Interest :  4779225
Time :  10:45 	 DATA :  1221900


Change

Change in Call Open Interest :  6286800
Change in Put Open Interest :  4584600
Change in Call Open Interest :  6286800
Change in Put Open Interest :  4584600
Time :  12:05 	 DATA :  -1702200


Change in Call Open Interest :  6294225
Change in Put Open Interest :  4637250
Change in Call Open Interest :  6294225
Change in Put Open Interest :  4637250
Time :  12:07 	 DATA :  -1656975


Change in Call Open Interest :  6297150
Change in Put Open Interest :  4638525
Change in Call Open Interest :  6297150
Change in Put Open Interest :  4638525
Time :  12:09 	 DATA :  -1658625


Change in Call Open Interest :  6405825
Change in Put Open Interest :  4706700
Change in Call Open Interest :  6405825
Change in Put Open Interest :  4706700
Time :  12:11 	 DATA :  -1699125


Change in Call Open Interest :  6374100
Change in Put Open Interest :  4794975
Change in Call Open Interest :  6374100
Change in Put Open Interest :  4794975
Time :  12:13 	 DATA :  -1579125


Change in Call Open Interest :  637

Change in Call Open Interest :  6201150
Change in Put Open Interest :  5495325
Change in Call Open Interest :  6201150
Change in Put Open Interest :  5495325
Time :  13:33 	 DATA :  -705825


Change in Call Open Interest :  6114000
Change in Put Open Interest :  5487150
Change in Call Open Interest :  6114000
Change in Put Open Interest :  5487150
Time :  13:35 	 DATA :  -626850


Change in Call Open Interest :  6114000
Change in Put Open Interest :  5482725
Change in Call Open Interest :  6114000
Change in Put Open Interest :  5482725
Time :  13:37 	 DATA :  -631275


Change in Call Open Interest :  6172575
Change in Put Open Interest :  5781225
Change in Call Open Interest :  6172575
Change in Put Open Interest :  5781225
Time :  13:39 	 DATA :  -391350


Change in Call Open Interest :  6292650
Change in Put Open Interest :  5895750
Change in Call Open Interest :  6292650
Change in Put Open Interest :  5895750
Time :  13:41 	 DATA :  -396900


Change in Call Open Interest :  6292650


Change in Call Open Interest :  3217125
Change in Put Open Interest :  7877175
Change in Call Open Interest :  3217125
Change in Put Open Interest :  7877175
Time :  15:03 	 DATA :  4660050


Change in Call Open Interest :  3217125
Change in Put Open Interest :  7877175
Change in Call Open Interest :  3056700
Change in Put Open Interest :  7769025
Time :  15:05 	 DATA :  4712325


Change in Call Open Interest :  3056700
Change in Put Open Interest :  7769025
Change in Call Open Interest :  3038775
Change in Put Open Interest :  7708575
Time :  15:07 	 DATA :  4669800


Change in Call Open Interest :  3038775
Change in Put Open Interest :  7708575
Change in Call Open Interest :  3049725
Change in Put Open Interest :  7836075
Time :  15:09 	 DATA :  4786350


Change in Call Open Interest :  3049725
Change in Put Open Interest :  7836075
Change in Call Open Interest :  3154950
Change in Put Open Interest :  7866000
Time :  15:11 	 DATA :  4711050


Change in Call Open Interest :  3154950
